In [1]:
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
yf.pdr_override() # <== thats all it takes :-)
!pip install monthly-returns-heatmap
import monthly_returns_heatmap as mrh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for monthly-returns-heatmap: filename=monthly_returns_heatmap-0.0.11-py3-none-any.whl size=8112 sha256=57246c9bdba1fc76757747e0b3d9680f1d8f96853807127106ab7403f505b31f
  Stored in directory: /root/.cache/pip/wheels/a8/68/93/e7887114a1f27ed095dbb50aed4d59bd66341e43a990725711
Successfully built monthly-returns-heatmap


In [206]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [268]:
df=pd.read_csv('/content/gdrive/MyDrive/NIFTYDATA/2023_June_Fifty_futures.csv')

df.reset_index(inplace=True, drop=True)

In [269]:
data=df.copy()

In [243]:
def get_adx(high, low, close, lookback):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()
    return plus_di, minus_di, adx_smooth

df['positive_DI'] = pd.DataFrame(get_adx(df['high'], df['low'], df['close'], 14)[0]).rename(columns = {0:'plus_di'})
df['negative_DI'] = pd.DataFrame(get_adx(df['high'], df['low'], df['close'], 14)[1]).rename(columns = {0:'minus_di'})
df['adx'] = pd.DataFrame(get_adx(df['high'], df['low'], df['close'], 14)[2]).rename(columns = {0:'adx'})
df = df.dropna()
df.tail()

df.reset_index(drop=True,inplace=True)

In [244]:
df.head(2)

,datetime,symbol,open,high,low,close,volume,positive_DI,negative_DI,adx
0,2022-05-04 13:15:00,NSE:NIFTY1!,16890.05,16993.55,16760.05,16809.60,3316750.0,16.571706,42.957693,54.035011
1,2022-05-04 14:15:00,NSE:NIFTY1!,16810.75,16863.00,16643.20,16724.25,4161100.0,14.858663,49.645935,49.355589


In [245]:
df['Dif_adx']=df['adx'].diff()
df=df.dropna().reset_index(drop=True)

In [247]:


def determine_signals(data):
    signals = ['Hold']
    position = 0  # Position in the market: 0 for out, 1 for long, -1 for short

    for i in range(1, len(data)):
        if data['positive_DI'][i] > data['negative_DI'][i] and data['Dif_adx'][i] >= 0.1 and position !=1 :
          signals.append('Buy')
          position = 1

        elif data['positive_DI'][i] < data['negative_DI'][i] and data['Dif_adx'][i] <= -0.1 and position != -1:
          signals.append('Sell')
          position = -1

        else:
          signals.append('Hold')

    return signals

trading_signals = determine_signals(df)

df[ "signals" ]= trading_signals
df['signals'].value_counts()
df[df['datetime'].str.contains('2023-06')].tail(50)

,datetime,symbol,open,high,low,close,volume,positive_DI,negative_DI,adx,Dif_adx,signals
1935,2023-06-14 15:15:00,NSE:NIFTY1!,18819.00,18822.00,18808.30,18808.30,404400.0,32.388763,17.978416,19.850748,0.676108,Hold
1936,2023-06-15 09:15:00,NSE:NIFTY1!,18804.00,18844.10,18782.80,18839.55,1378800.0,37.625398,24.373431,20.439532,0.588784,Hold
1937,2023-06-15 10:15:00,NSE:NIFTY1!,18839.85,18867.50,18817.40,18818.15,835650.0,38.431745,21.159897,20.545141,0.105609,Hold
1938,2023-06-15 11:15:00,NSE:NIFTY1!,18818.10,18820.00,18770.00,18789.15,1064250.0,32.998738,28.970759,21.033180,0.488040,Hold
1939,2023-06-15 12:15:00,NSE:NIFTY1!,18789.15,18802.60,18775.00,18784.45,427500.0,29.583216,25.972151,19.995092,-1.038089,Hold
1940,2023-06-15 13:15:00,NSE:NIFTY1!,18783.05,18820.85,18761.00,18764.15,803050.0,29.692237,25.648253,19.035272,-0.959819,Hold
1941,2023-06-15 14:15:00,NSE:NIFTY1!,18761.55,18769.65,18734.40,18741.20,1395000.0,27.369760,29.120977,18.176105,-0.859167,Sell
1942,2023-06-15 15:15:00,NSE:NIFTY1!,18741.30,18749.95,18739.00,18739.00,500500.0,25.168535,26.778909,17.099241,-1.076864,Hold
1943,2023-06-16 09:15:00,NSE:NIFTY1!,18774.95,18839.00,18771.05,18830.65,1307650.0,38.401746,22.990326,16.211557,-0.887684,Hold
1944,2023-06-16 10:15:00,NSE:NIFTY1!,18831.00,18849.00,18825.90,18831.80,585400.0,38.149008,21.691792,16.858916,0.647359,Buy


In [248]:
df['signals'].value_counts()

df=df[['datetime','signals','close']]
df=df[df['signals'].str.contains('Buy|Sell')].reset_index(drop=True)

PNL = [0]
for i in range(1,len(df)-1):
  if df['signals'][i] == "Buy":
    money = (df['close'][i+1] - df['close'][i]) * 50
  else:
    money = (df['close'][i] - df['close'][i+1]) * 50
  PNL.append(money)
PNL.append(0)


Hold    1902
Sell      42
Buy       41
Name: signals, dtype: int64

In [255]:
df['PNL'] = PNL

In [258]:
df.tail(50)

,datetime,signals,close,PNL
33,2022-10-04 10:15:00,Buy,17247.00,-8755.0
34,2022-10-10 09:15:00,Sell,17071.90,-11640.0
35,2022-10-14 10:15:00,Buy,17304.70,-4095.0
36,2022-10-17 09:15:00,Sell,17222.80,-13597.5
37,2022-10-18 10:15:00,Buy,17494.75,-2652.5
38,2022-10-20 09:15:00,Sell,17441.70,-9465.0
39,2022-10-21 09:15:00,Buy,17631.00,24950.0
40,2022-11-03 09:15:00,Sell,18130.00,-3107.5
41,2022-11-07 09:15:00,Buy,18192.15,-5307.5
42,2022-11-10 09:15:00,Sell,18086.00,-14362.5


In [257]:
df['PNL'].sum()

59320.00000000108

# PART 2 : Using talib

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib



In [280]:
import talib

In [281]:
df=pd.read_csv('/content/gdrive/MyDrive/NIFTYDATA/2023_June_Fifty_futures.csv')

df.reset_index(inplace=True, drop=True)

In [282]:
df['adx']=talib.ADX(df['high'],df['low'],df['close'],timeperiod=14)
df['positive_DI']=talib.PLUS_DI(df['high'],df['low'],df['close'],timeperiod=14)
df['negative_DI']=talib.MINUS_DI(df['high'],df['low'],df['close'],timeperiod=14)

In [283]:
df['Dif_adx']=df['adx'].diff()
df=df.dropna().reset_index(drop=True)

In [284]:


def determine_signals(data):
    signals = ['Hold']
    position = 0  # Position in the market: 0 for out, 1 for long, -1 for short

    for i in range(1, len(data)):
        if data['positive_DI'][i] > data['negative_DI'][i] and data['Dif_adx'][i] >= 0.1 and position !=1 :
          signals.append('Buy')
          position = 1

        elif data['positive_DI'][i] < data['negative_DI'][i] and data['Dif_adx'][i] <= -0.1 and position != -1:
          signals.append('Sell')
          position = -1

        else:
          signals.append('Hold')

    return signals

trading_signals = determine_signals(df)

df[ "signals" ]= trading_signals
df['signals'].value_counts()
df[df['datetime'].str.contains('2023-06')].tail(50)

,datetime,symbol,open,high,low,close,volume,adx,positive_DI,negative_DI,Dif_adx,signals
1922,2023-06-14 15:15:00,NSE:NIFTY1!,18819.00,18822.00,18808.30,18808.30,404400.0,25.033899,28.031717,12.825226,0.937305,Hold
1923,2023-06-15 09:15:00,NSE:NIFTY1!,18804.00,18844.10,18782.80,18839.55,1378800.0,24.582089,24.308816,16.646639,-0.451810,Hold
1924,2023-06-15 10:15:00,NSE:NIFTY1!,18839.85,18867.50,18817.40,18818.15,835650.0,24.845564,26.652971,14.904398,0.263475,Hold
1925,2023-06-15 11:15:00,NSE:NIFTY1!,18818.10,18820.00,18770.00,18789.15,1064250.0,23.219274,23.958028,22.982827,-1.626289,Hold
1926,2023-06-15 12:15:00,NSE:NIFTY1!,18789.15,18802.60,18775.00,18784.45,427500.0,21.709148,22.599624,21.679716,-1.510126,Hold
1927,2023-06-15 13:15:00,NSE:NIFTY1!,18783.05,18820.85,18761.00,18764.15,803050.0,20.891372,23.522560,19.144775,-0.817777,Hold
1928,2023-06-15 14:15:00,NSE:NIFTY1!,18761.55,18769.65,18734.40,18741.20,1395000.0,19.579496,21.898478,23.033048,-1.311876,Sell
1929,2023-06-15 15:15:00,NSE:NIFTY1!,18741.30,18749.95,18739.00,18739.00,500500.0,18.361325,21.404099,22.513055,-1.218170,Hold
1930,2023-06-16 09:15:00,NSE:NIFTY1!,18774.95,18839.00,18771.05,18830.65,1307650.0,19.142897,33.694762,18.422632,0.781572,Buy
1931,2023-06-16 10:15:00,NSE:NIFTY1!,18831.00,18849.00,18825.90,18831.80,585400.0,20.051368,34.109706,17.625953,0.908471,Hold


In [288]:
df['signals'].value_counts()


Sell    45
Buy     44
Name: signals, dtype: int64

In [285]:

df=df[['datetime','signals','close']]
df=df[df['signals'].str.contains('Buy|Sell')].reset_index(drop=True)

PNL = [0]
for i in range(1,len(df)-1):
  if df['signals'][i] == "Buy":
    money = (df['close'][i+1] - df['close'][i]) * 50
  else:
    money = (df['close'][i] - df['close'][i+1]) * 50
  PNL.append(money)
PNL.append(0)


In [286]:
df['PNL'] = PNL

In [291]:
df.to_csv("ADX_DI_test.csv",index=False)

In [292]:
from google.colab import files
files.download('ADX_DI_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [289]:
sum(df['PNL'])

-46502.499999999694